<a href="https://colab.research.google.com/github/crodier1/machine_learning_deep_learning/blob/main/Portuguese_Tutor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Read this first!

Make sure you get your Open AI API key first. Put it under secrets with the name "OPENAI_API_KEY".

In [9]:
from openai import OpenAI
from google.colab import userdata
import gradio as gr
from io import BytesIO
from pydub import AudioSegment
from IPython.display import Audio, display
import threading

In [10]:
open_api_key = userdata.get('OPENAI_API_KEY')

openai = OpenAI(api_key=open_api_key)
MODEL = 'gpt-4o-mini'

In [11]:
system_message = """
You are a friendly and patient Brazilian Portuguese tutor. Your goal is to help students practice conversational Brazilian Portuguese in a natural, engaging way.
- Speak primarily in Brazilian Portuguese, using English only when necessary for clarity. Use a Brazilian Portuguese accent.
- Show genuine interest in what the student says and keep the conversation flowing with follow-up questions unless the student is saying goodbye.
- If the student makes grammatical or vocabulary errors, gently correct them by:
  1. Rephrasing their sentence correctly,
  2. Explaining the rule briefly,
  3. Giving a simple example for practice.
- Adjust your questions to the student’s level, gradually increasing complexity as they improve.
- Incorporate cultural references, idiomatic expressions, and everyday scenarios to make the conversation authentic.
- Always encourage and motivate the student to continue speaking, even after mistakes.
"""


In [12]:
def talker(message):
    response = openai.audio.speech.create(
        model="tts-1",
        voice="onyx",
        input=message
    )
    audio_content = response.content

    display(Audio(audio_content, rate=24000, autoplay=True))

In [13]:
# talker("Ola, tudo bem. Chamo-me, Cristiano.")

In [14]:
def chat(message, history):

  messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

  response = openai.chat.completions.create(model=MODEL, messages=messages)

  choice = response.choices[0].message

  threading.Thread(target=talker, args=(choice.content,), daemon=True).start()

  return choice.content

In [ ]:
opening_message = "Olá, chamo-me Cristiano. Sou seu professor de português brasileiro. Sobre o que você quer falar?"

threading.Thread(target=talker, args=(opening_message,), daemon=True).start()

initial_messages = [
    {"role": "assistant", "content": opening_message}
]

def transcribe_audio(audio_file):
    if audio_file is None:
        return ""
    try:
        with open(audio_file, "rb") as file:
            # Assuming primary language is Portuguese, but Whisper can handle English too.
            transcript = openai.audio.transcriptions.create(
                model="whisper-1",
                file=file,
                response_format="text",
                language="pt"
            )
        return transcript # Removed .text as transcript is already a string with response_format="text"
    except Exception as e:
        print(f"Error during transcription: {e}")
        return f"Erro na transcrição: {e}"


with gr.Blocks() as demo:
    chatbot = gr.Chatbot(value=initial_messages, type="messages", height=400, label="Seu Tutor de Português")

    with gr.Row():
        audio_input = gr.Audio(
            sources=["microphone"],
            type="filepath",
            label="Gravar áudio (Português/Inglês)",
            streaming=False,
            scale=1
        )
        msg = gr.Textbox(
            label="Sua mensagem",
            placeholder="Digite sua mensagem ou grave um áudio...",
            scale=3
        )
        send_btn = gr.Button("Enviar", scale=0)

    # When audio is recorded, transcribe it and put it into the text input box
    audio_input.change(
        fn=transcribe_audio,
        inputs=[audio_input],
        outputs=[msg]
    )

    # Function to handle user input and update the chatbot
    def user_message_wrapper(user_input, history_state):
        if not user_input:
            return "", history_state # Do not send empty messages

        # Add user message to history
        new_history = history_state + [{"role": "user", "content": user_input}]

        # Get assistant's response using the chat function
        response_content = chat(user_input, history_state)

        # Add assistant's response to history
        new_history.append({"role": "assistant", "content": response_content})

        # Clear the message input box and return the updated history
        return "", new_history

    # Bind send button and textbox submit to the wrapper function
    send_btn.click(
        fn=user_message_wrapper,
        inputs=[msg, chatbot],
        outputs=[msg, chatbot],
        scroll_to_output=True
    )
    msg.submit(
        fn=user_message_wrapper,
        inputs=[msg, chatbot],
        outputs=[msg, chatbot],
        scroll_to_output=True
    )

demo.launch(debug=True)

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://60a53d91e8156d365d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
